# Part One


As you finally start making your way upriver, you realize your pack is much lighter than you remember. <br>Just then, one of the items from your pack goes flying overhead. Monkeys are playing Keep Away with your missing things!

To get your stuff back, you need to be able to predict where the monkeys will throw your items.<br> After some careful observation, you realize the monkeys operate based on how worried you are about each item.

You take some notes (your puzzle input) on the items each monkey currently has, how worried you are about those items, <br>and how the monkey makes decisions based on your worry level. For example:


Each monkey has several attributes:<br>

Starting items lists your worry level for each item the monkey is currently holding in the order they will be inspected.<br>
Operation shows how your worry level changes as that monkey inspects an item.<br> (An operation like new = old * 5 means that your worry level after the monkey inspected the item is <br>five times whatever your worry level was before inspection.)
Test shows how the monkey uses your worry level to decide where to throw an item next.<br>
If true shows what happens with an item if the Test was true.<br>
If false shows what happens with an item if the Test was false.<br>
After each monkey inspects an item but before it tests your worry level, your relief that the <br>monkey's inspection didn't damage the item causes your worry level to be divided by three and rounded down to the nearest integer.

The monkeys take turns inspecting and throwing items. On a single monkey's turn, it <br>inspects and throws all of the items it is holding one at a time and in the order listed. Monkey 0 goes first, <br>then monkey 1, and so on until each monkey has had one turn. The process of each monkey taking a single turn is called a round.

When a monkey throws an item to another monkey, the item goes on the end of the <br>recipient monkey's list. A monkey that starts a round with no items could end up inspecting and throwing many items by the time its turn comes around. If a monkey is holding no items at the start of its turn, its turn ends.<br>

Chasing all of the monkeys at once is impossible; you're going to have to focus on the <br>two most active monkeys if you want any hope of getting your stuff back. Count the total number of times each monkey inspects items over 20 rounds:

Monkey 0 inspected items 101 times.<br>
Monkey 1 inspected items 95 times.<br>
Monkey 2 inspected items 7 times.<br>
Monkey 3 inspected items 105 times.<br>
In this example, the two most active monkeys inspected items 101 and 105 times. <br>The level of monkey business in this situation can be found by multiplying these together: 10605.

Figure out which monkeys to chase by counting how many items they inspect over 20 rounds. <br>What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans?

In [227]:
import numpy as np
import re
import tqdm

In [228]:
path = "../data/puzzle_11.txt"

In [229]:
with open(path, 'r') as file:
  # Read the contents of the file into a list
  data = file.readlines()

In [230]:
data = [x.replace("\n", "") for x in data]

In [202]:
class Monkey:
    def __init__(self, monkey_id, starting_items, operation, test, true_action, false_action, modulo_value=None):
        self.monkey_id = monkey_id
        self.items = starting_items
        self.operation = operation
        self.worry_level = None
        self.test = test
        self.true_monkey = true_action
        self.false_monkey = false_action
        self.activity = 0
        self.modulo_value = modulo_value
        
    def run_operation(self, old):
        statement = self.operation.replace("old", str(old))
        return eval(statement)
    
    def run_test(self, value):
        return value % self.test == 0
    
    def play_with_item(self, item_worry):
        new_item_worry = self.run_operation(item_worry)
        if self.modulo_value is not None:
            new_item_worry = int(new_item_worry % self.modulo_value)
        else:
            new_item_worry = int(np.floor(new_item_worry / 3))
        self.activity += 1
        if self.run_test(new_item_worry):
            return self.true_monkey, new_item_worry
        else:
            return self.false_monkey, new_item_worry

In [205]:
def extract_numbers(string):
    # Use a regular expression to find all numbers in the string
    matches = re.finditer(r'\d+', string)

    # Extract the numbers from the matches
    numbers = [int(match.group()) for match in matches]
    return numbers

In [206]:
def parse_monkeys(data: list, idx: int, modulo_value=None):
    info = data[idx: idx+6]
    monkey_id = extract_numbers(info[0])[0]
    starting_items = extract_numbers(info[1])
    operation = info[2].split("=")[1]
    test = extract_numbers(info[3])[0]
    monkey_true = extract_numbers(info[4])[0]
    monkey_false = extract_numbers(info[5])[0]
    monkey = Monkey(monkey_id, starting_items, operation, test, monkey_true, monkey_false, modulo_value)
    return monkey

In [207]:
def setup(data, modulo_value=None):
    monkey_starts = [i for i, x in enumerate(data) if "Monkey" in x]
    monkeys = []
    for i in monkey_starts:
        monkeys.append(parse_monkeys(data, i, modulo_value))
    for m in monkeys:
        print(f"Monkey {m.monkey_id} has {m.items}")
    return monkeys

In [215]:
def play_round(monkeys):
    for monkey in monkeys:
        for item in monkey.items:
            dst_monkey_id, new_item = monkey.play_with_item(item)
            dst_monkey = [m for m in monkeys if m.monkey_id == dst_monkey_id][0]
            # print(f"{monkey.monkey_id} passed {item} to {dst_monkey.monkey_id}")
            dst_monkey.items.append(new_item)
        monkey.items = []
    worries = list(chain(*[m.items for m in monkeys]))
    return monkeys

In [217]:
def play_game(data, rounds, use_modulo=False):
    if use_modulo:
        mod_value = np.prod([extract_numbers(x)[0] for x in data if "Test" in x])
        monkeys = setup(data, modulo_value=mod_value)
    else:
        monkeys = setup(data)
    for _ in tqdm.tqdm(range(rounds)):
        monkeys = play_round(monkeys)
    print("Game completed!")
    for m in monkeys:
        print(f"Monkey {m.monkey_id} has {m.items} and activity -> {m.activity}")
    activities = [m.activity for m in monkeys]
    activities.sort(reverse=True)
    return activities[0] * activities[1], monkeys

In [220]:
mb, monkeys = play_game(data, 20, False)
mb

Monkey 0 has [79, 98]
Monkey 1 has [54, 65, 75, 74]
Monkey 2 has [79, 60, 97]
Monkey 3 has [74]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 4428.58it/s]

Game completed!
Monkey 0 has [10, 12, 14, 26, 34] and activity -> 101
Monkey 1 has [245, 93, 53, 199, 115] and activity -> 95
Monkey 2 has [] and activity -> 7
Monkey 3 has [] and activity -> 105


10605

# Part Two

Worry levels are no longer divided by three after each item is inspected; you'll need to find another way to keep your worry levels manageable. <br>Starting again from the initial state in your puzzle input, what is the level of monkey business after 10000 rounds?

In [226]:
mb, monkeys = play_game(data, 10000, True)
mb

Monkey 0 has [54, 82, 90, 88, 86, 54]
Monkey 1 has [91, 65]
Monkey 2 has [62, 54, 57, 92, 83, 63, 63]
Monkey 3 has [67, 72, 68]
Monkey 4 has [68, 89, 90, 86, 84, 57, 72, 84]
Monkey 5 has [79, 83, 64, 58]
Monkey 6 has [96, 72, 89, 70, 88]
Monkey 7 has [79]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:05<00:00, 1850.50it/s]

Game completed!
Monkey 0 has [3622294, 6973096, 696826, 696826, 4877801, 821567, 313241, 313241, 313241, 4877801, 4193117, 313241, 8757677, 313241, 7080404] and activity -> 118588
Monkey 1 has [2192691, 5045541, 2192691, 2192691, 2898123, 2192691, 9039531, 2898123] and activity -> 60301
Monkey 2 has [4877786, 5956682, 313226, 313226, 4877786] and activity -> 60417
Monkey 3 has [6864832, 9105616, 7393906, 7393906, 7393906] and activity -> 60268
Monkey 4 has [] and activity -> 60273
Monkey 5 has [5956691, 5956691, 4877795] and activity -> 120704
Monkey 6 has [] and activity -> 120642
Monkey 7 has [] and activity -> 60483


14561971968